# Imports and Setup

In [5]:
%load_ext autoreload
%autoreload 2
    
# Necessary/extra dependencies. 
import shutil

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# ClearML
from clearml import Task
import dotenv
dotenv.load_dotenv()
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
!echo $CLEARML_API_ACCESS_KEY
!echo $CLEARML_API_SECRET_KEY

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
L1Z1RFJFCBKLMXAD05YQ
oSnY7ZetYphpyydowL8FMMNKqnc2npGn67Ls0rz8ECwAAkQYR9


Move `data.yaml` in the `yolov5/data` directory.

In [12]:
%ls

CITATION.cff     __pycache__/   export.py         runs/           val.py
CONTRIBUTING.md  benchmarks.py  hubconf.py        segment/        yolov5s.pt
LICENSE          classify/      models/           train.py
README.md        data/          pyproject.toml    tutorial.ipynb
README.zh-CN.md  detect.py      requirements.txt  utils/


In [3]:
shutil.copy('./data/data.yaml', './yolov5/data')

'./yolov5/data/data.yaml'

# 2.Train

Train a YOLOv5s model on the custom dataset starting from pretrained `--weights yolov5s.pt`. Training results are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.

In [9]:
%cd yolov5/data
%ls ../../data/train
%cd ..
%ls

/home/default/yolov5/data
ls: cannot access '../../data/train': No such file or directory
/home/default/yolov5
CITATION.cff     __pycache__/   export.py         runs/           val.py
CONTRIBUTING.md  benchmarks.py  hubconf.py        segment/        yolov5s.pt
LICENSE          classify/      models/           train.py
README.md        data/          pyproject.toml    tutorial.ipynb
README.zh-CN.md  detect.py      requirements.txt  utils/


In [3]:
task = Task.init(project_name='yolov5_v3', task_name='dataset_hole')

ClearML Task: created new task id=7d280bdf7f4f4925a17cefcc090741de
2024-05-15 16:36:18,813 - clearml.Repository Detection - WARNING - Jupyter Notebook auto-logging failed, could not access: /home/default/yolov5/yolo5-v3_fla.ipynb
2024-05-15 16:36:18,819 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/d6caf4b635544db7b78ff369d7217bff/experiments/7d280bdf7f4f4925a17cefcc090741de/output/log


In [4]:
!python3 train.py --img 640 --batch 2 --epochs 300 --data data/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cl

In [47]:
task.close()

# 3. Inference

We use the best weights for the inference

In [8]:
%ls

Dockerfile         data.yaml            main_train.py
Dockerfile_jetson  data_640_augmented/  requirements_.txt
LICENSE            data_black/          requirements_jetson.txt
README.md          data_holes/          yolo5-v3_fla.ipynb
cartone_edit.png   main_inference.py    yolov5/


In [13]:
!python3 detect.py --weights runs/train/exp20/weights/best.pt --img 640 --source ../data_holes/test/images --line-thickness 1 --save-txt

detect: weights=['runs/train/exp20/weights/best.pt'], source=../data_holes/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/160 /home/default/data_holes/test/images/c1_51_b8_a0_61_d8_23_rot_jpeg.rf.ea4ec98e017a9304df23d135e2a59fd1.jpg: 640x640 6 cutfolds, 1.5ms
image 2/160 /home/default/data_holes/test/images/c1_51_b8_a0_61_d8_7_rot_jpeg.rf.e8b8da616c64da8bae7051b7ca6ff794.jpg: 640x640 6 cutfolds, 1.7ms
image 3/16

In [49]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('runs/detect/exp9/*.jpeg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# inference

In [50]:
!python3 detect.py --weights runs/train/exp15/weights/best.pt --img 640 --source ./../cartone_edit.png --line-thickness 1 --save-txt

detect: weights=['runs/train/exp15/weights/best.pt'], source=./../cartone_edit.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/default/cartone_edit.png: 640x640 36 folds, 14 holes, 4 perforators, 1.5ms
Speed: 0.2ms pre-process, 1.5ms inference, 49.1ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp10
1 labels saved to runs/detect/exp10/labels
